###Import data in to colab 

In [2]:
!wget -q !wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip

In [3]:
! unzip /content/dataset52a7b21.zip

Archive:  /content/dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


###Import neccesary library

In [5]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import re
import os
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [6]:
contractions_dict={
    "I'm": "I am",
    "I'm'a": "I am about to",
    "I'm'o": "I am going to",
    "I've": "I have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'd": "I would",
    "I'd've": "I would have",
    "Whatcha": "What are you",
    "amn't": "am not",
    "ain't": "are not",
    "aren't": "are not",
    "'cause": "because",
    "can't": "can not",
    "can't've": "can not have",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "daren't": "dare not",
    "daresn't": "dare not",
    "dasn't": "dare not",
    "didn't": "did not",
    "didn’t": "did not",
    "don't": "do not",
    "don’t": "do not",
    "doesn't": "does not",
    "e'er": "ever",
    "everyone's": "everyone is",
    "finna": "fixing to",
    "gimme": "give me",
    "gon't": "go not",
    "gonna": "going to",
    "gotta": "got to",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he've": "he have",
    "he's": "he is",
    "he'll": "he will",
    "he'll've": "he will have",
    "he'd": "he would",
    "he'd've": "he would have",
    "here's": "here is",
    "how're": "how are",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how's": "how is",
    "how'll": "how will",
    "isn't": "is not",
    "it's": "it is",
    "'tis": "it is",
    "'twas": "it was",
    "it'll": "it will",
    "it'll've": "it will have",
    "it'd": "it would",
    "it'd've": "it would have",
    "kinda": "kind of",
    "let's": "let us",
    "luv": "love",
    "ma'am": "madam",
    "may've": "may have",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "ne'er": "never",
    "o'": "of",
    "o'clock": "of the clock",
    "ol'": "old",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "o'er": "over",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shalln't": "shall not",
    "shan't've": "shall not have",
    "she's": "she is",
    "she'll": "she will",
    "she'd": "she would",
    "she'd've": "she would have",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "somebody's": "somebody is",
    "someone's": "someone is",
    "something's": "something is",
    "sux": "sucks",
    "that're": "that are",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "that'd've": "that would have",
    "em": "them",
    "there're": "there are",
    "there's": "there is",
    "there'll": "there will",
    "there'd": "there would",
    "there'd've": "there would have",
    "these're": "these are",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they'd": "they would",
    "they'd've": "they would have",
    "this's": "this is",
    "those're": "those are",
    "to've": "to have",
    "wanna": "want to",
    "wasn't": "was not",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we'd": "we would",
    "we'd've": "we would have",
    "weren't": "were not",
    "what're": "what are",
    "what'd": "what did",
    "what've": "what have",
    "what's": "what is",
    "what'll": "what will",
    "what'll've": "what will have",
    "when've": "when have",
    "when's": "when is",
    "where're": "where are",
    "where'd": "where did",
    "where've": "where have",
    "where's": "where is",
    "which's": "which is",
    "who're": "who are",
    "who've": "who have",
    "who's": "who is",
    "who'll": "who will",
    "who'll've": "who will have",
    "who'd": "who would",
    "who'd've": "who would have",
    "why're": "why are",
    "why'd": "why did",
    "why've": "why have",
    "why's": "why is",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "you're": "you are",
    "you've": "you have",
    "you'll've": "you shall have",
    "you'll": "you will",
    "you'd": "you would",
    "you'd've": "you would have"
 }

In [7]:
import csv

###Read data with pandas

In [17]:
data =pd.read_csv("/content/dataset/train.csv",escapechar="\\",quoting=csv.QUOTE_NONE,nrows=5000)
data.shape

(5000, 5)

In [18]:
test=pd.read_csv("/content/dataset/test.csv",escapechar="\\",quoting=csv.QUOTE_NONE)
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [19]:
test['PRODUCT_ID']

0              1
1              2
2              3
3              4
4              5
           ...  
110770    110771
110771    110772
110772    110773
110773    110774
110774    110775
Name: PRODUCT_ID, Length: 110775, dtype: int64

In [20]:
!import nltk
nltk.download('punkt')

/bin/bash: import: command not found
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

###Check null values in datasets

In [22]:
data.isnull().sum()

TITLE                0
DESCRIPTION       1596
BULLET_POINTS      539
BRAND              253
BROWSE_NODE_ID       0
dtype: int64

In [23]:
test.isnull().sum()

PRODUCT_ID           0
TITLE                7
DESCRIPTION      10469
BULLET_POINTS     6786
BRAND             2430
dtype: int64

###Fill null value with zero

In [24]:
train_data=data.fillna(0)
test=test.fillna(0)
train_data.shape ,test.shape

((5000, 5), (110775, 5))

###Join all text data

In [25]:
cols = ['TITLE','DESCRIPTION','BULLET_POINTS']
train_data["text"] =train_data["TITLE"].astype(str) + train_data["DESCRIPTION"].astype(str) + train_data["BULLET_POINTS"].astype(str)
train_data.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID,text
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0,"Pete The Cat Bedtime Blues Doll, 14.5 InchPete..."
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ..."
2,The Ultimate Self-Sufficiency Handbook: A Comp...,0,Skyhorse Publishing,imusti,2,The Ultimate Self-Sufficiency Handbook: A Comp...
3,Amway Nutrilite Kids Chewable Iron Tablets (100),0,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3,Amway Nutrilite Kids Chewable Iron Tablets (10...
4,Teacher Planner Company A4 6 Lesson Academic T...,0,0,0,4,Teacher Planner Company A4 6 Lesson Academic T...


In [26]:
cols = ['TITLE','DESCRIPTION','BULLET_POINTS']
test["text"] =test["TITLE"].astype(str) + test["DESCRIPTION"].astype(str) + test["BULLET_POINTS"].astype(str)
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,text
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command,"Command 3M Small Kitchen Hooks, White, Decorat..."
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands,"NFL Detroit Lions Portable Party Fridge, 15.8 ..."
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic,Panasonic Single Line KX-TS880MX Corded Phone ...
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero,Zero Baby Girl's 100% Cotton Innerwear Bloomer...


###Drop columns

In [27]:
data=train_data.drop(cols, axis=1)
test=test.drop(cols, axis=1)
test.head()

,PRODUCT_ID,BRAND,text
0,1,Command,"Command 3M Small Kitchen Hooks, White, Decorat..."
1,2,O'Neal,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...
2,3,Boelter Brands,"NFL Detroit Lions Portable Party Fridge, 15.8 ..."
3,4,Panasonic,Panasonic Single Line KX-TS880MX Corded Phone ...
4,5,Zero,Zero Baby Girl's 100% Cotton Innerwear Bloomer...


####Text **processing**

In [28]:
import re
punc = r'[^\w\s]|^[a-zA-z]+|[a-zA-z]+"" "'
for ele in data:
	if ele in punc:
		data =data.replace(ele, "")

In [29]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [31]:
import string

In [32]:
##word tokenize
data['text'] = data['text'].apply(lambda x: word_tokenize(x)) 

test['text'] = test['text'].apply(lambda x: word_tokenize(x))

In [33]:
##function for normalize data
def normalize_tokens(list_of_tokens):
    return map(lambda x: x.lower(),list_of_tokens)

In [34]:
data['text'] = data['text'].apply(lambda x: normalize_tokens(x))

test['text'] = test['text'].apply(lambda x: normalize_tokens(x))

In [35]:
data['text'] = data['text'].apply(lambda x: list(x))

test['text'] = test['text'].apply(lambda x: list(x))

In [36]:
data.head()

,BRAND,BROWSE_NODE_ID,text
0,MerryMakers,0,"[pete, the, cat, bedtime, blues, doll, ,, 14.5..."
1,The New Yorker,1,"[the, new, yorker, nyhm014, refrigerator, magn..."
2,imusti,2,"[the, ultimate, self-sufficiency, handbook, :,..."
3,Amway,3,"[amway, nutrilite, kids, chewable, iron, table..."
4,0,4,"[teacher, planner, company, a4, 6, lesson, aca..."


In [37]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [1]:
##change in constraction word
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [39]:
def contractions_expansion(list_of_tokens):
    return map(contracted_word_expansion,list_of_tokens)

In [40]:
data['text'] = data['text'].apply(lambda x: contractions_expansion(x))
data['text'] = data['text'].apply(lambda x: list(x))

test['text'] = test['text'].apply(lambda x: contractions_expansion(x))
test['text'] = test['text'].apply(lambda x: list(x))

In [41]:
data.head()

,BRAND,BROWSE_NODE_ID,text
0,MerryMakers,0,"[pete, the, cat, bedtime, blues, doll, ,, 14.5..."
1,The New Yorker,1,"[the, new, yorker, nyhm014, refrigerator, magn..."
2,imusti,2,"[the, ultimate, self-sufficiency, handbook, :,..."
3,Amway,3,"[amway, nutrilite, kids, chewable, iron, table..."
4,0,4,"[teacher, planner, company, a4, 6, lesson, aca..."


##Regular expression for filter waste word

In [42]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [43]:
def waste_word_or_not(token):
    return re.search(regex,token)

In [44]:
def filter_waste_words(list_of_tokens):
    return filterfalse(waste_word_or_not,list_of_tokens)

In [45]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x))
data['text'] = data['text'].apply(lambda x: list(x))

test['text'] = test['text'].apply(lambda x: filter_waste_words(x))
test['text'] = test['text'].apply(lambda x: list(x))

In [46]:
data.head()

,BRAND,BROWSE_NODE_ID,text
0,MerryMakers,0,"[pete, the, cat, bedtime, blues, doll, inchpet..."
1,The New Yorker,1,"[the, new, yorker, refrigerator, magnet, x, ne..."
2,imusti,2,"[the, ultimate, handbook, a, complete, guide, ..."
3,Amway,3,"[amway, nutrilite, kids, chewable, iron, table..."
4,0,4,"[teacher, planner, company, lesson, academic, ..."


In [47]:
def split(list_of_tokens):
    return map(lambda x: re.split(regex,x)[0],list_of_tokens)

In [48]:
data['text'] = data['text'].apply(lambda x: split(x))
data['text'] = data['text'].apply(lambda x: list(x))

test['text'] = test['text'].apply(lambda x: split(x))
test['text'] = test['text'].apply(lambda x: list(x))

In [49]:
data.head()

,BRAND,BROWSE_NODE_ID,text
0,MerryMakers,0,"[pete, the, cat, bedtime, blues, doll, inchpet..."
1,The New Yorker,1,"[the, new, yorker, refrigerator, magnet, x, ne..."
2,imusti,2,"[the, ultimate, handbook, a, complete, guide, ..."
3,Amway,3,"[amway, nutrilite, kids, chewable, iron, table..."
4,0,4,"[teacher, planner, company, lesson, academic, ..."


In [50]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [52]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [53]:
def stopwords_removal(list_of_tokens):
    return filter(is_stopword,list_of_tokens)

In [54]:
data['text'] = data['text'].apply(lambda x: stopwords_removal(x))
data['text'] = data['text'].apply(lambda x: list(x))

test['text'] = test['text'].apply(lambda x: stopwords_removal(x))
test['text'] = test['text'].apply(lambda x: list(x))

In [55]:
data.head()

,BRAND,BROWSE_NODE_ID,text
0,MerryMakers,0,"[pete, cat, bedtime, blues, doll, inchpete, ca..."
1,The New Yorker,1,"[new, yorker, refrigerator, magnet, new, yorke..."
2,imusti,2,"[ultimate, handbook, complete, guide, baking, ..."
3,Amway,3,"[amway, nutrilite, kids, chewable, iron, table..."
4,0,4,"[teacher, planner, company, lesson, academic, ..."


In [56]:
data[train_data.BROWSE_NODE_ID==0].head()

,BRAND,BROWSE_NODE_ID,text
0,MerryMakers,0,"[pete, cat, bedtime, blues, doll, inchpete, ca..."
80,Barbie,0,"[barbie, fairytopia, magic, rainbow, mix, swit..."
429,0,0,"[barbie, fashionistas, barbie, doll]"
1106,Mattel,0,"[mattel, special, edition, chucke, cheeses, ba..."
1640,Burberry,0,"[barbie, doll, accessories, necklaces, earring..."


In [57]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [58]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [59]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
id_data=data['BROWSE_NODE_ID']

In [61]:
from collections import Counter

###Most frequent word in each row

In [62]:
tokenized_sents =data['text']
df = pd.DataFrame({"tokenized_sents": tokenized_sents})
final_df = pd.DataFrame({"KeyWords" : df["tokenized_sents"].apply(lambda x: [k for k, v in Counter(x).most_common(10)])}) 

tokenized_sents1 =test['text']
df = pd.DataFrame({"tokenized_sents1": tokenized_sents1})
test_df = pd.DataFrame({"KeyWords" : df["tokenized_sents1"].apply(lambda x: [k for k, v in Counter(x).most_common(3)])})

In [63]:
result = pd.concat([final_df,id_data], axis=1)

In [64]:
##convert list to string 
def listtostring(s):
    str1= " "
    return (str1.join(s))

In [65]:
result['KeyWords'] = result['KeyWords'].apply(lambda x: listtostring(x))
result

,KeyWords,BROWSE_NODE_ID
0,cat pete bedtime blues doll ages popular new r...,0
1,new yorker magnet handsome cello wrapped hard ...,1
2,ultimate handbook complete guide baking crafts...,2
3,tablets nutrilite kids chewable iron amway qua...,3
4,teacher planner company lesson academic leathe...,4
...,...,...
4995,mosquito room socket electric bug insect trap ...,4995
4996,set half cotton little toonyport sleeve length...,1003
4997,athlero pair style neck men slim fit wide rang...,5
4998,br case material designer soft phone protectio...,1045


In [66]:
test_df['KeyWords'] = test_df['KeyWords'].apply(lambda x: listtostring(x))
test_df

,KeyWords
0,hooks easy command
1,synthetic leather palm
2,home fridge boelter
3,features display redial
4,panties baby wear
...,...
110770,trimmer br shaver
110771,layer kg grin
110772,masks asian filtration
110773,face mask ply


In [67]:
result['BROWSE_NODE_ID']

0          0
1          1
2          2
3          3
4          4
        ... 
4995    4995
4996    1003
4997       5
4998    1045
4999    1010
Name: BROWSE_NODE_ID, Length: 5000, dtype: int64

In [68]:
x_train=result['KeyWords']

In [69]:
y_train=result['BROWSE_NODE_ID']

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report

In [71]:
vectorizer = CountVectorizer()
vectorizer.fit(result['KeyWords'])
vector = vectorizer.transform(result['KeyWords'])

In [72]:
vectorizer = CountVectorizer()
vectorizer.fit(test_df['KeyWords'])
vector1 = vectorizer.transform(test_df['KeyWords'])

In [73]:
from sklearn import preprocessing 
label = preprocessing.LabelEncoder() 
 
result['BROWSE_NODE_ID']= label.fit_transform(result['BROWSE_NODE_ID']) 
print(result['BROWSE_NODE_ID'].unique())

[   0    1    2 ... 1736 1737 1738]


In [74]:
print(vector.shape)

(5000, 10638)


In [75]:
print(vector1.shape)

(110775, 14966)


###Applied Tf-IDF to transform text into a meaninful representation of numbers

In [76]:
# extract the tfid representation matrix of the text data
tfidf_converter = TfidfTransformer()
X_tfidf = tfidf_converter.fit_transform(vector).toarray()
X_tfidf.shape

(5000, 10638)

In [77]:
# extract the tfid representation matrix of the text data
tfidf_converter = TfidfTransformer()
test_tfidf = tfidf_converter.fit_transform(vector1).toarray()
test_tfidf.shape

(110775, 14966)

In [78]:
test=pd.DataFrame(data=test_tfidf)

In [79]:
drop_col=np.arange(10638, 14966, 1)

In [80]:
x_test=test.drop(drop_col,axis=1)
x_test.to_numpy()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [81]:
y_train=result['BROWSE_NODE_ID']

###Model training using Randomforest

In [ ]:
from sklearn import linear_model
regr =RandomForestClassifier()
regr.fit(X_tfidf,y_train)

###Predict lables for testdata

In [ ]:
y_pred=regr.predict(x_test)
y_pred

In [ ]:
test_label=pd.DataFrame(y_pred)

In [ ]:
product_id=test['PRODUCT_ID']

###**Defining the key and values as per the hackathon**

In [ ]:
dataframe=pd.concat([product_id,test_label], axis=1)
dataframe

In [ ]:
dataframe.columns =['PRODUCT_ID','BROWSE_NODE_ID']

In [ ]:
dataframe

,PRODUCT_ID,BROWSE_NODE_ID
0,1,802
1,2,722
2,3,1436
3,4,1295
4,5,1109
...,...,...
110770,110771,4
110771,110772,245
110772,110773,642
110773,110774,1688


In [ ]:
dataframe.to_csv('submission.csv',index=False)

In [ ]:
dataframe

,PRODUCT_ID,BROWSE_NODE_ID
0,1,802
1,2,722
2,3,1436
3,4,1295
4,5,1109
...,...,...
110770,110771,4
110771,110772,245
110772,110773,642
110773,110774,1688
